In [155]:
from tensorflow import keras
from PIL import Image
from torchvision import models, transforms
import torch.nn as nn
import torch.optim as optim
import torch
import random
from torchvision import models, transforms
from numpy.linalg import norm
import numpy as np
import math

# Load Data

In [156]:
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
import torch

transform = transforms.Compose([
    transforms.Resize((224, 244)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), #Is transformed to be better with VGG16 model
    ])

In [157]:
def getDataset(train=True, sample_size=100):
    dataset = CIFAR10(root='./data',
                  train=train, 
                  download=True,
                  transform=transform)
    
    X = []
    y = []

    for i in range(sample_size):
        X.append( dataset[i][0] )
        y.append( dataset[i][1] )

    X = torch.stack( X )

    return X, y

X_train, y_train = getDataset() #Training data
X_test, y_test = getDataset(train=False) #Test data

Files already downloaded and verified
Files already downloaded and verified


# Create Minibatch

In [158]:
batchSize = 32

In [159]:
indices = np.random.choice(X_train.shape[0], batchSize, replace=False) # Not used as of now
minibatch = X_train[indices]
#minibatch

# Load Model

In [160]:
from torchvision import models
pretrained_model = models.vgg16(pretrained=True)
pretrained_model.classifier = torch.nn.Identity() # 25088 features output

import torch.nn as nn

test_model = nn.Sequential(  nn.Flatten(), #25088 -> 4000 features - 4000 is arbitrarily chosen to mimic the paper
                        nn.Linear(25088,4000),
                        nn.ReLU(),
                        nn.Linear(4000,4000),
                        nn.Sigmoid()
                        )

# Forward Propagate Through Model

In [161]:
result = test_model(pretrained_model(X_train))

In [162]:
result = result.cpu().detach().numpy() #(sample_size, 4000) shape

In [163]:
# Parameters
mean = 0
std_dev = np.sqrt(0.01)  # Standard deviation is the square root of the variance

# Create the matrix W and V with shape (4000, 32) and (32, 1) - SHOULD BE A FUNCTION WHERE 32 IS THE NUMBER OF BITS WE WANT
W = np.random.normal(mean, std_dev, (4000, 32))
V = np.random.normal(mean, std_dev, (32, 1)) 

W.shape
V.shape

(32, 1)

In [164]:
U = []
for i in range(len(result)):
    dot = np.dot(W.T, result[i])
    dot = dot.reshape(32, 1)
    u = (dot + V)
    U.append(u)
U = np.array(U) # (100, 32, 1) Shape -> (sample_size, #bits, 1) Shape

In [165]:
#Remove the 1 from (100, 32, 1)
U_reshaped = U.reshape(100, 32)

# Compute the dot product matrix -> Useful for finding Theta in loss function
dot_product_matrix = np.matmul(U_reshaped, U_reshaped.T)
dot_product_matrix #(100, 100) Shape -> (sample_size, sample_size) Shape

#Calculate Theta
Theta = 1/2 * dot_product_matrix

array([[230.50406818, 232.88130093, 233.39866353, ..., 234.83099668,
        237.60753017, 234.71841394],
       [232.88130093, 237.79267316, 237.06318261, ..., 238.34650701,
        241.28707649, 238.26974149],
       [233.39866353, 237.06318261, 237.78906568, ..., 238.72830097,
        241.44032997, 238.27245713],
       ...,
       [234.83099668, 238.34650701, 238.72830097, ..., 240.44086888,
        242.96230963, 239.58884761],
       [237.60753017, 241.28707649, 241.44032997, ..., 242.96230963,
        246.50481053, 242.50929525],
       [234.71841394, 238.26974149, 238.27245713, ..., 239.58884761,
        242.50929525, 240.79442923]])

# Finding S Based On Labels

In [36]:
y_train = np.array(y_train)
S = (y_train[:, None] == y_train).astype(int) # (sample_size, sample_size) Shape -> 1 if labels are the same, 0 if not

# Finding binary codes B

In [168]:
#Find the Binary codes
B = np.sign(U)
B

(100, 32, 1)

## CUSTOM LOSS IMPLEMENTATION

In [ ]:
#Parameters
eta = 0.25 #Learning Rate

In [7]:
import torch.nn as nn

class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()

    def forward(self):
        loss = - sum(S * Theta - math.log(1 + math.e^Theta)) + eta * sum(norm(B - U)^2)    
        return loss.mean()